<a href="https://colab.research.google.com/github/KunalRoutray/Deep-Learning-With-Tensorflow/blob/main/ASSIGNMENT_4(DLWTF_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

kunal routray

2341018202

**WORD EMBEDDING**

**Q1. Import text8 dataset of word2vec from gensim model and perform the following task.**

In [ ]:
!pip install gensim


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 26.7 MB/s eta 0:00:00


**1(a) Import the word vector model**

In [ ]:
import gensim.downloader as api
from gensim.models import Word2Vec
import os
info = api.info("text8")
assert(len(info) > 0)
dataset = api.load("text8")
model = Word2Vec(dataset)
# Create the data directory if it doesn't exist
if not os.path.exists("data"):
   os.makedirs("data")
model.save("data/text8-word2vec.bin")

from gensim.models import KeyedVectors
def print_most_similar(word_conf_pairs, k):
   for i, (word, conf) in enumerate(word_conf_pairs):
       print("{:.3f} {:s}".format(conf, word))
       if i >= k-1:
          break
       if k < len(word_conf_pairs):
          print("...")
model = KeyedVectors.load("data/text8-word2vec.bin")
word_vectors = model.wv
print(word_vectors)


[==================================================] 100.0% 31.6/31.6MB downloaded
KeyedVectors<vector_size=100, 71290 keys>


**1(b) Write Python code to print the vocabulary to show first 15 words.**

In [ ]:
words = word_vectors.key_to_index.keys() #stores all words seen during tra
print([x for i, x in enumerate(words) if i < 15])
assert("king" in words)
print("words similar to king")


['the', 'of', 'and', 'one', 'in', 'a', 'to', 'zero', 'nine', 'two', 'is', 'as', 'eight', 'for', 's']
words similar to king


**1(c) Write Python code to print the most similar words to “king”.**


In [ ]:
print_most_similar(word_vectors.most_similar("king"), 5) #returns list of t
print(" vector arithmetic with words (cosine similarity)")


0.710 prince
...
0.707 queen
...
0.705 emperor
...
0.701 kings
...
0.696 throne
 vector arithmetic with words (cosine similarity)


**1(d) Write Python code to print the most similar words to “king”.
d Write a Python code to perform an analogy test: France is to Paris? is to Berlin . Analogy
a− b + c
here, positive[a,c] , negative[b], which is equivalent to a is to b as c is to .**


In [ ]:

print(" france + berlin - paris = ?")
print_most_similar(word_vectors.most_similar(positive=["france", "berlin"], negative=["paris"]), 1)
print("vector arithmetic with words (Levy and Goldberg)")
print("france + berlin - paris = ?")
print_most_similar(word_vectors.most_similar_cosmul(
positive=["france", "berlin"], negative=["paris"]), 1
)

 france + berlin - paris = ?
0.799 germany
vector arithmetic with words (Levy and Goldberg)
france + berlin - paris = ?
0.976 germany


**1(e) Write a Python code to find odd one out from the list [”hindus”, ”parsis”, ”singapore”, ”christians”]**

In [ ]:
print("find odd one out")
print("[hindus, parsis, singapore, christians]")
print(word_vectors.doesnt_match(["hindus", "parsis", "singapore", "christian"]))


find odd one out
[hindus, parsis, singapore, christians]
singapore


**1(f) Write a Python code to find the similarity of man with woman, dog,whale and tree**

In [ ]:

print("similarity between words")
for word in ["woman", "dog", "whale", "tree"]:
    print("similarity({:s}, {:s}) = {:.3f}".format( "man", word, word_vectors.similarity("man", word)))

similarity between words
similarity(man, woman) = 0.735
similarity(man, dog) = 0.451
similarity(man, whale) = 0.273
similarity(man, tree) = 0.257


**1(g) Write a Python code to find similar by word to Singapore.**

In [ ]:
print("similar by word")
print_most_similar(word_vectors.similar_by_word("singapore"), 5)

similar by word
0.898 malaysia
...
0.842 uganda
...
0.836 philippines
...
0.833 indonesia
...
0.826 thailand


**1(h) Write a Python code to find the distance between two vectors like Singapore and ‘Malaysia**

In [ ]:
print("distance between vectors")
print("distance(singapore, malaysia) = {:.3f}".format(word_vectors.distance("singapore", "malaysia")))


distance between vectors
distance(singapore, malaysia) = 0.102


**Q2. Write Python code to find the most similar word for ‘machine’ from the given sentence.The Sentence is  “I love machine learning. Machine Learning is fun.”**

In [ ]:
from gensim.models import Word2Vec
sentences = [["i","love","machine","learning"],["machine","learning","is","fun"]]

# CBOW: sg=0 (skip-gram would be sg=1)
model = Word2Vec(
sentences,
vector_size=100,
# embedding dim
window=2,
min_count=1,
workers=4,
sg=0,
negative=10,
epochs=30
)
print("Vector for 'learning':", model.wv["learning"][:5])
print("Most similar to 'machine':", model.wv.most_similar("machine", topn=5))

Vector for 'learning': [-0.00053623  0.00023643  0.00510335  0.00900927 -0.00930295]
Most similar to 'machine': [('is', 0.06797364354133606), ('i', 0.009403552860021591), ('love', 0.004535813815891743), ('learning', -0.010849320329725742), ('fun', -0.023415347561240196)]


**Q3. Write Python code to implement CBoW architecture from scratch.For the setence “I love machine learning." find the nearest word for learning,machine and love. Use Word2Vec and CBoW to find the most similar words.**

In [ ]:
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.metrics.pairwise import cosine_similarity
raw_sentences = ["I love machine learning"]
def normalize(s):
   s = s.lower()
   s = re.sub(r"[^a-z\s]", " ", s)
   s = re.sub(r"\s+", " ", s).strip()
   return s
sentences = [normalize(s).split() for s in raw_sentences]
# 1) Build vocab
from collections import Counter
word_counts = Counter([w for sent in sentences for w in sent])
itos = sorted(word_counts.keys())
stoi = {w:i for i,w in enumerate(itos)}
V = len(stoi)
# 2) Build CBOW training pairs
# context window = W (left W, right W)
W = 2 # window size
contexts, targets = [], []
for sent in sentences:
    idxs = [stoi[w] for w in sent]
    for i, center in enumerate(idxs):
        ctx = []
        for k in range(1, W+1):
            if i-k >= 0: ctx.append(idxs[i-k])
            if i+k < len(idxs): ctx.append(idxs[i+k])
        if len(ctx) > 0: # allow edges (variable ctx length)
           # pad or crop to fixed 2*W so model input is fixed
           # simple strategy: if short, repeat ends
           while len(ctx) < 2*W:
                ctx.append(ctx[-1])
           if len(ctx) > 2*W:
              ctx = ctx[:2*W]
           contexts.append(ctx)
           targets.append(center)
contexts = np.array(contexts, dtype=np.int32) # shape (N, 2W)
targets = np.array(targets, dtype=np.int32) # shape (N,)
y_onehot = tf.keras.utils.to_categorical(targets, num_classes=V)
print(f"Vocab size: {V}, pairs: {len(targets)}, window: {W}")
print("Example context idxs:", contexts[0], "-> target idx:", targets[0])

# 3) Define CBOW model
class CBOWModel(tf.keras.Model):
      def __init__(self, vocab_sz, emb_sz, window_sz, **kwargs):
          super().__init__(**kwargs)
          self.window_sz = window_sz
          self.embedding = layers.Embedding(
               input_dim=vocab_sz,
               output_dim=emb_sz,
               embeddings_initializer="glorot_uniform",
               mask_zero=False, # we padded by repetition, not zer
               name="embedding"
          )
          self.dense = layers.Dense(
             vocab_sz,
             kernel_initializer="glorot_uniform",
             activation="softmax",
             name="softmax_head"
           )
      def call(self, x):
          x = self.embedding(x)
          x = tf.reduce_mean(x, axis=1) # (batch, emb)
          x = self.dense(x) # (batch, V)
          return x
VOCAB_SIZE = V
EMBED_SIZE = 100
WINDOW_SIZE = W
model = CBOWModel(VOCAB_SIZE, EMBED_SIZE, WINDOW_SIZE)
model.build(input_shape=(None, WINDOW_SIZE * 2))
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
loss="categorical_crossentropy",
metrics=["accuracy"])
model.summary()

# 4) Train
history = model.fit(contexts, y_onehot, batch_size=32,
epochs=100,
verbose=0,
validation_split=0.2
)
print(f"Final train acc: {history.history['accuracy'][-1]:.3f} | "
f"val acc: {history.history['val_accuracy'][-1]:.3f}")

# 5) Extract embeddings
emb_weights = model.get_layer("embedding").get_weights()[0] # (V, EMBED_SI
print("Embedding matrix shape:", emb_weights.shape)

# 6) Utility: most similar words (cosine)
def most_similar(query, topk=5):
    if query not in stoi:
       return []
    qv = emb_weights[stoi[query]][None, :]
    sims = cosine_similarity(qv, emb_weights)[0]
    order = np.argsort(-sims)
    out = []
    for idx in order:
        w = itos[idx]
        if w == query:
           continue
        out.append((w, float(sims[idx])))
        if len(out) >= topk:
           break
    return out
print("\nNearest to 'learning':", most_similar("learning"))
print("Nearest to 'machine':", most_similar("machine"))
print("Nearest to 'love':", most_similar("love"))


Vocab size: 4, pairs: 4, window: 2
Example context idxs: [2 3 3 3] -> target idx: 0


/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'cbow_model', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "cbow_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax_head (Dense)            │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Final train acc: 1.000 | val acc: 0.000
Embedding matrix shape: (4, 100)

Nearest to 'learning': [('i', 0.3010336756706238), ('love', -0.05245177447795868), ('machine', -0.07170701026916504)]
Nearest to 'machine': [('love', 0.01448216661810875), ('learning', -0.07170701026916504), ('i', -0.2994571328163147)]
Nearest to 'love': [('i', 0.021397151052951813), ('machine', 0.01448216661810875), ('learning', -0.05245177447795868)]


**Q4. Download GloVe embedding and find the most similar word to king using cosine similarity.**


In [ ]:
# Download GloVe embedding
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip


--2026-02-10 09:44:26--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2026-02-10 09:44:27--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  4.89MB/s    in 2m 43s  

2026-02-10 09:47:11 (5.03 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflatin

In [ ]:
#find the most similar word to king using cosine similarity.
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
GLOVE_PATH = "/content/glove.6B.100d.txt" # change path if needed
word2vec = {}
with open(GLOVE_PATH, 'r', encoding='utf-8') as f:
  for line in tqdm(f, total=400000): # 400k vocab in glove.6B
      parts = line.strip().split()
      word = parts[0]
      vec = np.array(parts[1:], dtype=np.float32)
      word2vec[word] = vec
print(f"Loaded {len(word2vec)} words. Vector size = {len(next(iter(word2vec.values())))}")

# 2. Define function to find most similar word
def most_similar(word, topk=5):
  if word not in word2vec:
    print(f"'{word}' not in vocabulary.")
    return []
  q_vec = word2vec[word].reshape(1, -1)
  all_words = list(word2vec.keys())
  all_vectors = np.array(list(word2vec.values()))
  # cosine similarity
  sims = cosine_similarity(q_vec, all_vectors)[0]
  # sort descending (excluding the word itself)
  top_idx = np.argsort(-sims)[1:topk+1]
  return [(all_words[i], float(sims[i])) for i in top_idx]

print("\nMost similar to 'king':")
print(most_similar("king", topk=10))


100%|██████████| 400000/400000 [00:09<00:00, 43284.48it/s]


Loaded 400000 words. Vector size = 100

Most similar to 'king':
[('prince', 0.7682329416275024), ('queen', 0.7507690191268921), ('son', 0.7020888924598694), ('brother', 0.6985775232315063), ('monarch', 0.6977890729904175), ('throne', 0.6919990181922913), ('kingdom', 0.6811410188674927), ('father', 0.6802029609680176), ('emperor', 0.6712857484817505), ('ii', 0.6676074266433716)]


**Q5. Write a Python code to compute the similarity between “man” and other words
(“woman”, “dog”, “whale”, “tree”) using GloVe embeddings**

In [ ]:
def similarity(word1, word2):
  if word1 not in word2vec or word2 not in word2vec:
    return None
  v1 = word2vec[word1].reshape(1, -1)
  v2 = word2vec[word2].reshape(1, -1)
  return float(cosine_similarity(v1, v2)[0][0])
target = "man"
others = ["woman", "dog", "whale", "tree"]
print(f"\nCosine similarity with '{target}':\n")
for w in others:
    sim = similarity(target, w)
    print(f"{target:>5s} - {w:<6s} = {sim:.4f}")


Cosine similarity with 'man':

  man - woman  = 0.8323
  man - dog    = 0.5643
  man - whale  = 0.3055
  man - tree   = 0.4319


**Q6. Write a Python code to train a sentence using GloVe embedding from scratch.Print the token,build the co-occurance matrix,initialize the vector and print the final vector after training the model for 3000 epochs.**


In [ ]:
from collections import Counter, defaultdict
import math
import numpy as np # numpy was imported later, moving it up for consistency
sentences = [
     "I love machine learning",
     "I love deep learning"
]
# Tokenize & vocab
sentences = [s.lower().split() for s in sentences]
vocab = sorted(set(word for s in sentences for word in s))
print(vocab)
stoi = {w:i for i,w in enumerate(vocab)}
print(stoi)
itos = {i:w for w,i in stoi.items()}
print(itos)
# Build co-occurrence matrix (window=1)
X = defaultdict(float)
window = 1
for sentence in sentences:
  for i, word in enumerate(sentence):
    for j in range(max(i-window,0), min(i+window+1, len(sentence))):
      if i != j:
        X[(stoi[word], stoi[sentence[j]])] += 1.0
print(X)
# Initialize vectors
dim = 2
W = np.random.randn(len(vocab), dim) * 0.1
bias = np.zeros(len(vocab))
lr = 0.05
# Train
for epoch in range(3000):
  loss = 0
  for (i, j), xij in X.items():
    target = math.log(xij)
    pred = np.dot(W[i], W[j]) + bias[i] + bias[j]
    diff = pred - target
    loss += diff**2
    # Gradient update
    grad = 2 * diff
    W[i] -= lr * grad * W[j]
    W[j] -= lr * grad * W[i]
    bias[i] -= lr * grad
    bias[j] -= lr * grad
  if epoch % 1000 == 0:
     print(f"epoch {epoch} loss {loss}")
for w in vocab:
    print(w, W[stoi[w]])



['deep', 'i', 'learning', 'love', 'machine']
{'deep': 0, 'i': 1, 'learning': 2, 'love': 3, 'machine': 4}
{0: 'deep', 1: 'i', 2: 'learning', 3: 'love', 4: 'machine'}
defaultdict(<class 'float'>, {(1, 3): 2.0, (3, 1): 2.0, (3, 4): 1.0, (4, 3): 1.0, (4, 2): 1.0, (2, 4): 1.0, (3, 0): 1.0, (0, 3): 1.0, (0, 2): 1.0, (2, 0): 1.0})
epoch 0 loss 0.7977490251701381
epoch 1000 loss 1.1955218030417551e-12
epoch 2000 loss 2.6525525390057237e-21
deep [ 0.10881816 -0.09966217]
i [-0.17165491  0.09709506]
learning [ 0.00995003 -0.0390901 ]
love [-0.10620272  0.16225377]
machine [-0.05768578 -0.19571621]


**Q7. Python code to load GloVe Word Vectors, each each line from the glove fi
and create an embedding matrix for the model and create a non-trainable keras embedding layer**

In [ ]:
import numpy as np
from tensorflow.keras.layers import Embedding
from tensorflow.keras.initializers import Constant
from tensorflow.keras.preprocessing.text import Tokenizer # Import Tokenizer
# Path to GloVe file (e.g., glove.6B.100d.txt)
glove_path = "/content/glove.6B.100d.txt"
# 1. Load GloVe vectors
embeddings_index = {}
with open(glove_path, encoding="utf-8") as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = vector
print(f"Loaded {len(embeddings_index)} word vectors from GloVe.")
embedding_dim = 100 # or 50/200/300 based on GloVe file used

word_index = {"the": 1, "a": 2, "of": 3, "to": 4, "in": 5, "and": 6, "king": 7, "prince": 8, "queen": 9, "man":10, "woman":11}

vocab_size = len(word_index) + 1

#2. Create embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
  vec = embeddings_index.get(word)
  if vec is not None:
     embedding_matrix[i] = vec
print("Embedding matrix shape:", embedding_matrix.shape)
print(embedding_matrix)
# 3. Define Keras Embedding layer (non-trainable)
embedding_layer = Embedding(input_dim=vocab_size,
                            output_dim=embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            trainable=False)
print("\nKeras Embedding layer created successfully.")

Loaded 400000 word vectors from GloVe.
Embedding matrix shape: (12, 100)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.038194   -0.24487001  0.72812003 ... -0.1459      0.82779998
   0.27061999]
 [-0.27085999  0.044006   -0.02026    ... -0.4923      0.63687003
   0.23642001]
 ...
 [-0.50045002 -0.70826     0.55387998 ... -0.36032     0.13347
  -0.56075001]
 [ 0.37292999  0.38503     0.71086001 ... -0.93711001  0.039138
  -0.53911   ]
 [ 0.59368002  0.44825     0.59320003 ... -0.54648     0.15162
  -0.30754   ]]

Keras Embedding layer created successfully.


**Q8. Write python code to perform spam detection using word embedding.**

In [ ]:
# 1. Dataset Simulation
data = [
    ("URGENT! You have won a 1 week FREE membership in our prize draw. Txt 'CLAIM' to 81020.", 1), # Spam
    ("Hey dad, I'm stuck in traffic, running late for dinner.", 0), # Ham
    ("Congratulations! You've been selected for a ␖2000 holiday voucher. Text YES to 88000 now!", 1), # Spam
    ("Call me later to discuss the project update. I need your input.", 0), # Ham
    ("WINNER! Claim your FREE iPhone 15 by texting 'FREE' to 60900. Hurry up!", 1), # Spam
    ("Ok thanks. See you tomorrow.", 0) # Ham
]
texts_8 = [d[0] for d in data]
labels_8 = np.array([d[1] for d in data])

# Configuration
MAX_WORDS = 1000 # Max number of words to keep in the vocabulary
MAX_LEN = 20 # Max sequence length
EMBEDDING_DIM_8 = 50 # Using 50 for simplicity
GLOVE_EMBEDDING_DIM = 100 # Defining GLOVE_EMBEDDING_DIM
# 2. Tokenization and Sequencing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer_8 = Tokenizer(num_words=MAX_WORDS)
tokenizer_8.fit_on_texts(texts_8)
sequences_8 = tokenizer_8.texts_to_sequences(texts_8)
padded_sequences_8 = pad_sequences(sequences_8, maxlen=MAX_LEN, padding='post', truncating='post')
word_index_8 = tokenizer_8.word_index
VOCAB_SIZE_8 = len(word_index_8) + 1

# 3. Embedding Matrix Creation (Using simulated GloVe, similar to Task 7)
embedding_matrix_8 = np.zeros((VOCAB_SIZE_8, EMBEDDING_DIM_8))
if GLOVE_EMBEDDING_DIM >= EMBEDDING_DIM_8:
  print(f" Creating embedding matrix for {VOCAB_SIZE_8} words, initializing from GloVe.")
  for word, i in word_index_8.items():
    # Fallback if the word is not in the small dummy GLOVE_VECTORS
    if i < VOCAB_SIZE_8 and i > 0:
      embedding_matrix_8[i] = np.random.rand(EMBEDDING_DIM_8) # Mock vector initialization

# 4. Model Definition
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense # Added Flatten
model_8 = Sequential([
      Embedding(
             input_dim=VOCAB_SIZE_8,
             output_dim=EMBEDDING_DIM_8,
             input_length=MAX_LEN,
             trainable=True
      ),
      Flatten(),
      Dense(24, activation='relu'),
      Dense(1, activation='sigmoid') # Binary classification
])
model_8.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("\n Spam Detection Model Summary (Keras):")
model_8.summary()

# 5. Training (Using the simulated tiny dataset)
print("\n Training the Spam Detection Model (Simulated):")
history = model_8.fit(
    padded_sequences_8,
    labels_8,
    epochs=10,
    verbose=0
)

# 6. Evaluation
loss, accuracy = model_8.evaluate(padded_sequences_8, labels_8, verbose=0)
print(f" Final training accuracy (on small data): {accuracy*100:.2f}%")


 Creating embedding matrix for 63 words, initializing from GloVe.

 Spam Detection Model Summary (Keras):


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


 Training the Spam Detection Model (Simulated):
 Final training accuracy (on small data): 100.00%


In [ ]:
#7. Prediction Example
new_spam_text = ["Free gift card! Click here now to claim."]
new_sequences = tokenizer_8.texts_to_sequences(new_spam_text)
new_padded = pad_sequences(new_sequences, maxlen=MAX_LEN, padding='post', truncating='post')
prediction = model_8.predict(new_padded, verbose=0)[0][0]
label = "SPAM" if prediction > 0.5 else "HAM"
print(f" Test prediction for '{new_spam_text[0]}': {prediction:.4f} -> {label}")


 Test prediction for 'Free gift card! Click here now to claim.': 0.3733 -> HAM
